In [1]:
from openmesh import *
import numpy as np
import os
from tqdm import tqdm

In [2]:
# FILE_PATH = os.getcwd()+"./data"
FILE_PATH = os.path.join(os.getcwd(), 'data', 'coseg', 'coseg_aliens', 'train', '1.obj')
FILE_PATH

'/home/pascal/3D_slAIcer/data/coseg/coseg_aliens/train/1.obj'

In [35]:
vtx_prop = VPropHandle()


NameError: name 'VPropHandle' is not defined

In [38]:

# mesh = om.read_trimesh(FILE_PATH, vertex_tex_coord=False, face_color=False)

# mesh = TriMesh()
mesh = TriMesh()
mesh.request_halfedge_normals()
mesh.request_vertex_normals()
mesh.request_vertex_texcoords3D()

# options = Options()
# options += Options.VertexNormal

mesh = read_trimesh(FILE_PATH)

# for vh in mesh.vertices():
#     print(vh.idx())

# mesh.calc_face_normal()

# mesh.request_vertex_normals()
# for vh in mesh.vertices():
#     print(vh.)


In [110]:

class ObjGeometry:

    def __init__(self):
        self.v = []
        self.f = []
        self.e = []
    
    def getFaceVerts(self, face):
        faceVerts = np.zeros((3, 3))
        faceVerts[0] = self.v[face[0]-1]  # -1 because start index is 0
        faceVerts[1] = self.v[face[1]-1]
        faceVerts[2] = self.v[face[2]-1]
        return faceVerts
    
    def findMatchingVerts(self, f1Id, f2Id):
        #Common values between 2 arrays
        return np.intersect1d(self.f[f1Id], self.f[f2Id])

    def findEdges(self):
        face1Id = 0
        for i in tqdm(range(len(self.f)), desc='Edge finding'):
            face1 = self.f[i]
            # face1Verts = self.getFaceVerts(face1)
            face2Id = 0
            for face2 in self.f:
                if face1Id != face2Id:
                    matches = self.findMatchingVerts(face1Id, face2Id)
                    if matches.size == 2:
                        edge = {
                            # Edge between these 2 vertex ids
                            'v': matches,
                            # Edge between these 2 face ids
                            'f': np.array([face1Id+1, face2Id+1])
                        }
                        self.e.append(edge)
                face2Id += 1
            face1Id += 1
        print(f'Done! {len(self.e)} edges found')
        return self.e
    
    def findAngleVertex(self, triangle, edge):
        edgeVert1Id = edge[0]
        edgeVert2Id = edge[1]
        # get triangle vertex that isn't on the edge
        mask = np.logical_and(triangle != edgeVert1Id, triangle != edgeVert2Id)
        print(f'Angle vertex: {triangle[mask]}')
        return triangle[mask][0]
    
    def calculateAdjacentVecs(self, angleVertId, edge):
        edgeVert1Id = edge[0]
        edgeVert2Id = edge[1]
        vec1 = self.v[angleVertId-1] - self.v[edgeVert1Id-1]
        vec2 = self.v[angleVertId-1] - self.v[edgeVert2Id-1]
        print(
            f'Vector between verts {angleVertId} ({self.v[angleVertId-1]}) and {edgeVert1Id} ({self.v[edgeVert1Id-1]}): {vec1}')
        print(
            f'Vector between verts {angleVertId} ({self.v[angleVertId-1]}) and {edgeVert2Id} ({self.v[edgeVert2Id-1]}): {vec2}')
        # return vec1, vec2
        return np.array([vec1, vec2])

    def calculateInnerAngles(self, thisEdge, angleVertIds, neighborTriangles, vecs):
        innerAngles = np.zeros((2,))

        for i, t in enumerate(neighborTriangles):

            # angleVertId = self.findAngleVertex(t, thisEdge)
            angleVertId = angleVertIds[i]
            # vec1, vec2 = self.calculateAdjacentVecs(angleVertId, thisEdge)
            vec1 = vecs[i][0]
            vec2 = vecs[i][1]
     
            theta = np.degrees(np.arccos(vec1.dot(vec2)/(np.linalg.norm(vec1) * np.linalg.norm(vec2))))
            print(f'theta = {theta}')
            innerAngles[i] = theta
            print('---------------')
        return innerAngles
    
    def calculateEdgeLenRatio(self, edge, angleVertIds, vecs):
        edgeVert1Id = edge[0]
        edgeVert2Id = edge[1]
        print('Calculating edge ratios')
        # print(f'edge: {edge}')
        # print(f'Ange vert Ids: {angleVertIds}')
        edgeLen = np.linalg.norm(self.v[edgeVert1Id-1] - self.v[edgeVert2Id-1])
        edgeMidVert = np.zeros((3,))

        edgeMidVert = np.array(
            [(self.v[edgeVert1Id-1][0] + self.v[edgeVert2Id-1][0])/2,
             (self.v[edgeVert1Id-1][1] + self.v[edgeVert2Id-1][1])/2,
             (self.v[edgeVert1Id-1][2] + self.v[edgeVert2Id-1][2])/2])
             
        print(f'EdgeLen : {edgeLen}')
        # print(f'EdgeMidVert: {edgeMidVert}')
        # Only need one vec (hypthenuse)
        a = np.linalg.norm(vecs[0][0]) #side 1 for 1st triangle
        c = np.linalg.norm(vecs[0][1]) #Hypotenuse 2 for 1st triangle
        
        b = edgeLen
        print(f'a: {a}, b: {b}, c:{c}')
        s = (a+b+c/2)
        A = np.sqrt(s*(s-a)*(s-b)*(s-c)) #Heron's formula
        h1 = A*2/b
        print(f'h1 : {h1}')

        a = np.linalg.norm(vecs[1][0])  # side 1 for 2nd triangle
        c = np.linalg.norm(vecs[1][1])  # side 2 for 2nd triangle
        # b = edgeLen
        s = (a+b+c/2)
        A = np.sqrt(s*(s-a)*(s-b)*(s-c))  # Heron's formula
        h2 = A*2/b
        print(f'a: {a}, b: {b}, c:{c}')
        print(f'h2 : {h2}')

        # h1 = np.linalg.norm()
        # h1 = np.linalg.norm(edgeMidVert-self.v[angleVertIds[0]-1])
        # h2 = np.linalg.norm(edgeMidVert-self.v[angleVertIds[1]-1])

        # print(f'h1: {h1}, h2: {h2}')

        return np.array([edgeLen/h1, edgeLen/h2])

        # edgeMidVert[0] = (self.v[edgeVert1Id][0] + self.v[edgeVert2Id][0])/2
        # edgeMidVert[1] = (self.v[edgeVert1Id][1] + self.v[edgeVert2Id][1])/2
        # edgeMidVert[2] = (self.v[edgeVert1Id][2] + self.v[edgeVert2Id][2])/2
        # calculate edge len
        # calculate triangle height
    
    def calculateFeatures(self, edges):
        # Caculate dihedral angle between 2 faces

        #Calculate inner angle of triangles
        i = 0
        print(edges[i])
        #Find dihedral of first triangle
        t1Id = edges[i]['f'][0]
        t2Id = edges[i]['f'][1]

        t1 = self.f[t1Id-1] #Index 0 for python list
        t2 = self.f[t2Id-1]

        print(f'Triangle {t1Id}: {t1}')
        print(f'Triangle {t2Id}: {t2}')

        neighborTriangles = [t1, t2]
        thisEdge = [edges[i]['v'][0], edges[i]['v'][1]]
        angleVertIds = []
        angleVertIds.append(self.findAngleVertex(t1, thisEdge))
        angleVertIds.append(self.findAngleVertex(t2, thisEdge))

        # vecs = np.zeros((2, 2,3))
        vecs = []
        vecs.append(self.calculateAdjacentVecs(angleVertIds[0], thisEdge)) #For 1st triangle
        vecs.append(self.calculateAdjacentVecs(angleVertIds[1], thisEdge)) #For 2nd triangle
        # vecs[0] = self.calculateAdjacentVecs(angleVertIds[0], thisEdge)
        # vecs[1] = self.calculateAdjacentVecs(angleVertIds[1], thisEdge)

        # vec1, vec2 = self.calculateAdjacentVecs(angleVertId, thisEdge)
        ia = self.calculateInnerAngles(thisEdge, angleVertIds, neighborTriangles, vecs)  # Inner angles
        elr = self.calculateEdgeLenRatio(thisEdge, angleVertIds, vecs)
        print(f'Inner angles: {ia}')
        print(f'Edge length ratios: {elr}')
       
        
        # calculate edge ratios



class Obj2verts:
    # def __init__(self):
    def readFile(self, filePath):
        geom = ObjGeometry()
        lines = []
        verts = []
        faces = []
        with open(filePath) as f:
            lines = f.readlines()

        for line in lines:
            line = line.strip().split(' ')
            if line[0] == 'v':
                geom.v.append(
                    np.array([line[1], line[2], line[3]], dtype=np.float))
                # verts.append(np.array([line[1], line[2], line[3]], dtype=np.float))
            elif line[0] == 'f':
                geom.f.append(
                    np.array([line[1], line[2], line[3]], dtype=np.int))
                # faces.append(np.array([line[1], line[2], line[3]], dtype=np.int))
        return geom


In [37]:


o2v = Obj2verts()
# v, f = o2v.readFile(FILE_PATH)
geometry = o2v.readFile(FILE_PATH)

# geometry.f[0]
geometry.findEdges()
geometry.calculateFeatures()
# geometry.calculateFeatures(edges)
# edges



Edge finding:   6%|▌         | 84/1500 [00:02<00:43, 32.62it/s]


KeyboardInterrupt: 

In [111]:
geom1 = o2v.readFile(FILE_PATH)
geom1.calculateFeatures(edges)


{'v': array([119, 247]), 'f': array([1, 2])}
Triangle 1: [247   9 119]
Triangle 2: [247 119 485]
Angle vertex: [9]
Angle vertex: [485]
Vector between verts 9 ([-211.951996   22.238701  140.240005]) and 119 ([-211.945999   24.3248    140.296997]): [-0.005997 -2.086099 -0.056992]
Vector between verts 9 ([-211.951996   22.238701  140.240005]) and 247 ([-217.195007   22.238701  140.240005]): [5.243011 0.       0.      ]
Vector between verts 485 ([-217.115997   24.3248    140.296997]) and 119 ([-211.945999   24.3248    140.296997]): [-5.169998  0.        0.      ]
Vector between verts 485 ([-217.115997   24.3248    140.296997]) and 247 ([-217.195007   22.238701  140.240005]): [0.07901  2.086099 0.056992]
theta = 90.16464879542359
---------------
theta = 92.1682052300429
---------------
Calculating edge ratios
EdgeLen : 5.648640731178523
a: 2.0868859791263166, b: 5.648640731178523, c:5.243010999999996
h1 : 16.07969642130521
a: 5.1699979999999925, b: 5.648640731178523, c:2.0883725017259267
h2

In [70]:
edges

[{'v': array([119, 247]), 'f': array([1, 2])},
 {'v': array([  9, 247]), 'f': array([  1, 483])},
 {'v': array([  9, 119]), 'f': array([  1, 651])},
 {'v': array([119, 247]), 'f': array([2, 1])},
 {'v': array([119, 485]), 'f': array([2, 3])},
 {'v': array([247, 485]), 'f': array([2, 6])},
 {'v': array([119, 485]), 'f': array([3, 2])},
 {'v': array([393, 485]), 'f': array([3, 4])},
 {'v': array([119, 393]), 'f': array([   3, 1082])},
 {'v': array([393, 485]), 'f': array([4, 3])},
 {'v': array([392, 485]), 'f': array([4, 6])},
 {'v': array([392, 393]), 'f': array([   4, 1085])},
 {'v': array([247, 392]), 'f': array([5, 6])},
 {'v': array([ 51, 247]), 'f': array([  5, 485])},
 {'v': array([ 51, 392]), 'f': array([  5, 552])},
 {'v': array([247, 485]), 'f': array([6, 2])},
 {'v': array([392, 485]), 'f': array([6, 4])},
 {'v': array([247, 392]), 'f': array([6, 5])},
 {'v': array([237, 286]), 'f': array([7, 8])},
 {'v': array([ 27, 237]), 'f': array([  7, 288])},
 {'v': array([ 27, 286]), 'f